# Training ML algos on EPH. Predicting on CENSO.

In [1]:
# Load modules
# import time
# time.sleep(2000)

import pandas as pd
import numpy as np


# Column names
y_cols = ['CAT_OCUP', 'P47T', 'PP10E', 'PP10D', 'PP07K', 'PP07I', 'V3_M', 'PP07G4', 'CH16', 'T_VI', 
          'V12_M', 'TOT_P12', 'PP07G3', 'V5_M', 'PP07H', 'V2_M', 'PP10C', 
          'PP08D1', 'PP07J', 'CAT_INAC', 'CH07', 'CH08', 'P21', 'PP07G1', 'PP07G_59', 'PP07G2']


x_cols = ['IX_TOT', 'P02', 'P03', 'AGLOMERADO', 'V01', 'H05', 'H06',
       'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H16', 'H15', 'PROP', 'H14',
       'H13', 'P07', 'P08', 'P09', 'P10', 'P05', 'CONDACT']

# Ahora si!
## Train on Quarterly data

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
# import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [3]:

# T = '20162'
# # Load EPH
# # PBA_train = pd.read_csv('./PBA_train.csv').fillna(0)
# PBA_train = pd.read_csv('./PBA_train_'+T+'.csv').fillna(0)
# # PBA_train = PBA_train.sample(frac = 1).reset_index(drop = True)
# # PBA_train.head()

# # Load Census
# # X_censo = pd.read_csv('./sample_censo_table025_A_bolivar.csv').fillna(0)
# # X_censo = pd.read_csv('./sample_censo_table025_A_vlopez_rodriguez.csv').fillna(0)
# X_censo = pd.read_csv('./sample_censo_table025rand.csv', nrows=300000, usecols = ['DPTO','RADIO_REF_ID','PERSONA_REF_ID','IX_TOT', 'P02', 'P03', 'CONDACT', 'AGLOMERADO', 'V01', 'H05', 'H06',
#        'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H16', 'H15', 'PROP', 'H14',
#        'H13', 'P07', 'P08', 'P09', 'P10', 'P05']).fillna(0).rename(columns = {'CONDACT': 'CA'})
# X_censo[x_cols].values.shape

In [12]:
# Load Census
# file_ = './../../Desktop/extracted_/sample_censo_table_f.1BA.csv'
file_ = 'sintetico_2017.1.csv'

X_censo = pd.read_csv(file_, usecols = ['DPTO','RADIO_REF_ID','PERSONA_REF_ID','IX_TOT', 'P02', 'P03', 'CONDACT', 'AGLOMERADO', 'V01', 'H05', 'H06',
       'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H16', 'H15', 'PROP', 'H14',
       'H13', 'P07', 'P08', 'P09', 'P10', 'P05', 'circuito', 'dpto_merge']).fillna(0)
X_censo[x_cols].values.shape

X_censo['P03'] = (abs(X_censo[['P03']] -19) < 3).astype(int) * X_censo[['P03']] + \
(abs(X_censo[['P03']] -19) >= 3).astype(int) * np.floor(10*X_censo[['P03']].rank(pct = 1)-0.01).astype(int)
X_censo['CONDACT'] = 100*X_censo['CONDACT']
X_censo['AGLOMERADO'] = 10*X_censo['AGLOMERADO']

In [13]:
X_censo.shape

(1527885, 29)

In [14]:
# Load EPH

PBA_train = pd.read_csv('./../../Desktop/extracted_/PBA_train.csv').fillna(0)
PBA_train['P03'] = (abs(PBA_train[['P03']] -19) < 3).astype(int) * PBA_train[['P03']] + \
(abs(PBA_train[['P03']] -19) >= 3).astype(int) * np.floor(10*PBA_train[['P03']].rank(pct = 1)-0.01).astype(int)
PBA_train['CONDACT'] = 100*PBA_train['CONDACT']
PBA_train['AGLOMERADO'] = 10*PBA_train['AGLOMERADO']

# PBA_train = PBA_train.loc[PBA_train.AGLOMERADO.isin(X_censo['AGLOMERADO'].unique())]


In [15]:
# PBA_train

In [16]:

### STEP 1
x_cols1 = x_cols

#     predecir = ['CAT_OCUP', 'CAT_INAC', 'CH07']
predecir1 = ['CAT_OCUP', 'CAT_INAC', 'CH07']
y_cols1 = predecir1

X = PBA_train[x_cols1]
y = PBA_train[y_cols1].loc[X.index]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
# Rewrite Census data as 'test' set.

X = X_train; y = y_train

# clf = tree.DecisionTreeClassifier()
clf1 = RandomForestClassifier(n_estimators=50)
clf1 = clf1.fit(X.values, y.values)

y_out1 = clf1.predict(X_censo[x_cols1].values)
y_censo_fit1 = pd.DataFrame(y_out1, index = X_censo.index, columns=predecir1)
Xy1_censo = pd.concat([X_censo, y_censo_fit1], axis = 1)

### STEP 2

x_cols2 = x_cols1+predecir1
# La seccion PP07G pregunta si el trabajo es en blanco y que beneficios tiene. Puede ayudar a la regresion para ingresos.
predecir2 = ['PP07G1', 'PP07G2', 'PP07G3', 'PP07G4', 'PP07G_59', 'PP07H', 'PP07I', 'PP07J', 'PP07K']
y_cols2 = predecir2
X = PBA_train[x_cols2]
y = PBA_train[y_cols2].loc[X.index]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
X = X_train; y = y_train

# clf2 = tree.DecisionTreeClassifier()
# clf2 = clf2.fit(X.values, y.values)
clf2 = RandomForestClassifier(n_estimators=50).fit(X.values, y.values)

y_out2 = clf2.predict(Xy1_censo[x_cols2].values)
y_censo_fit2 = pd.DataFrame(y_out2, index = X_censo.index, columns=predecir2)
Xy2_censo = pd.concat([Xy1_censo, y_censo_fit2], axis = 1)


In [17]:
pd.options.display.max_columns = 99
Xy2_censo

,RADIO_REF_ID,V01,DPTO,circuito,H05,H06,H07,H08,H09,H10,H11,H12,H13,H14,H15,H16,PROP,IX_TOT,AGLOMERADO,PERSONA_REF_ID,P02,P03,P05,P07,P08,P09,P10,CONDACT,dpto_merge,CAT_OCUP,CAT_INAC,CH07,PP07G1,PP07G2,PP07G3,PP07G4,PP07G_59,PP07H,PP07I,PP07J,PP07K
0,8774,1,6490,583,2,4.0,2,1,1.0,1,2,3,1.0,2.0,3,4,1,1,330,8785286,2,6,2,1,2,2,2,300,lomas de zamora,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8726,1,6490,583,2,5.0,2,2,1.0,1,1,2,1.0,2.0,3,5,1,1,330,8716767,2,1,1,1,2,2,1,300,lomas de zamora,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8698,1,6490,583,1,3.0,1,1,1.0,1,1,2,2.0,2.0,1,2,1,1,330,8682579,2,7,1,1,2,2,1,100,lomas de zamora,3.0,0.0,5.0,2.0,2.0,2.0,2.0,5.0,2.0,2.0,1.0,4.0
3,8738,1,6490,583,1,4.0,1,1,1.0,1,1,3,1.0,1.0,2,2,1,1,330,8732245,1,3,1,1,2,6,1,100,lomas de zamora,3.0,0.0,5.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
4,8756,1,6490,583,1,2.0,2,1,1.0,1,1,3,1.0,1.0,2,3,1,1,330,8757281,1,18,1,1,1,4,2,300,lomas de zamora,0.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,8668,1,6490,583,2,4.0,2,1,1.0,1,1,3,1.0,1.0,3,4,1,1,330,8642779,2,3,1,1,2,4,1,100,lomas de zamora,3.0,0.0,5.0,2.0,2.0,2.0,2.0,0.0,2.0,0.0,1.0,4.0
6,8290,1,6490,583,1,2.0,1,1,1.0,1,1,3,2.0,2.0,2,3,1,1,330,8276578,2,8,1,1,2,2,2,300,lomas de zamora,0.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8302,1,6490,583,2,5.0,2,1,1.0,1,1,2,1.0,1.0,2,2,4,1,330,8294732,2,18,1,1,1,5,2,300,lomas de zamora,0.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,8324,1,6490,583,1,9.0,1,1,1.0,1,1,2,1.0,1.0,3,4,1,1,330,8319013,2,4,1,1,2,9,0,300,lomas de zamora,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,8733,1,6490,583,1,2.0,1,3,1.0,1,2,3,1.0,2.0,1,2,1,1,330,8725547,2,3,1,1,2,2,2,300,lomas de zamora,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# save RFC_25rand_20162
Xy2_censo.to_csv('sintetico_circus_condact.csv', index = False)

In [19]:
for T in ['20162', '20163', '20164', '20171', '20172',  '20173',  '20174',  '20181', '20182', '20183']:
    print(T)
    ### STEP 3 (Regression)
    PBA_train = pd.read_csv('./../../Desktop/extracted_/PBA_train_'+T+'.csv').fillna(0)

    x_cols3 = x_cols2 + predecir2
    # Columnas de ingresos. Necesitan una regresion...
    predecir3 = ['P47T', 'V3_M', 'T_VI', 'V12_M', 'TOT_P12', 'V5_M','V2_M', 'PP08D1', 'P21']
    y_cols3 = predecir3

    X = PBA_train[x_cols3].sample(frac = 1) #PBA_train_reg
    y = PBA_train[y_cols3].loc[X.index] #PBA_train_reg

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    X = X_train; y = y_train

    clf3 = RandomForestRegressor(n_estimators=50)
    clf3 = clf3.fit(X.values, y.values)

    y_out3 = clf3.predict(Xy2_censo[x_cols3].values)
    y_censo_fit3 = pd.DataFrame(y_out3, index = X_censo.index, columns=predecir3)
    Xy3_censo = pd.concat([Xy2_censo, y_censo_fit3], axis = 1)
    Xy3_censo['ANO4'] = T[:-1]; Xy3_censo['TRIMESTRE'] = T[-1]

    
    # save
#     Xy3_censo.to_csv('RFReg_.01AR'+T+'.csv', index = False)
    Xy3_censo.to_csv('sintetico_circus_income_YQ'+T+'.csv', index = False)

20162
20163
20164
20171
20172
20173
20174
20181
20182
20183
